<a href="https://colab.research.google.com/github/Joseph-Willem-Ricci/5450-final-project/blob/master/CIS5450_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Libraries
import pandas as pd

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/

In [ ]:
!!kaggle datasets download -d dhruvildave/spotify-charts

['Downloading spotify-charts.zip to /content',
 '',
 '  0% 0.00/945M [00:00<?, ?B/s]',
 '  0% 1.00M/945M [00:00<08:20, 1.98MB/s]',
 '  0% 3.00M/945M [00:00<02:58, 5.55MB/s]',
 '  1% 7.00M/945M [00:00<01:13, 13.3MB/s]',
 '  1% 11.0M/945M [00:00<00:50, 19.5MB/s]',
 '  1% 14.0M/945M [00:01<00:53, 18.3MB/s]',
 '  2% 17.0M/945M [00:01<00:45, 21.2MB/s]',
 '  2% 22.0M/945M [00:01<00:34, 28.2MB/s]',
 '  3% 26.0M/945M [00:01<00:31, 30.7MB/s]',
 '  3% 31.0M/945M [00:01<00:28, 34.2MB/s]',
 '  4% 35.0M/945M [00:01<00:26, 36.2MB/s]',
 '  4% 40.0M/945M [00:01<00:23, 40.3MB/s]',
 '  5% 45.0M/945M [00:01<00:26, 35.5MB/s]',
 '  5% 50.0M/945M [00:02<00:24, 39.0MB/s]',
 '  6% 54.0M/945M [00:02<00:26, 35.9MB/s]',
 '  6% 58.0M/945M [00:02<00:49, 18.7MB/s]',
 '  6% 61.0M/945M [00:02<00:45, 20.4MB/s]',
 '  7% 66.0M/945M [00:02<00:35, 26.0MB/s]',
 '  8% 71.0M/945M [00:03<00:32, 28.0MB/s]',
 '  8% 75.0M/945M [00:03<00:33, 27.6MB/s]',
 '  9% 81.0M/945M [00:03<00:26, 34.3MB/s]',
 '  9% 85.0M/945M [00:03<00:27, 3

In [ ]:
!unzip /content/spotify-charts.zip

Archive:  /content/spotify-charts.zip
  inflating: charts.csv              


In [ ]:
#Read the csv file and save it to a dataframe 
df_charts =[]
df_charts = pd.read_csv('charts.csv')
# Check out the first five rows
df_charts.head(5)

,title,rank,date,artist,url,region,chart,trend,streams
0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,top200,SAME_POSITION,253019.0
1,Vente Pa' Ca (feat. Maluma),2,2017-01-01,Ricky Martin,https://open.spotify.com/track/7DM4BPaS7uofFul...,Argentina,top200,MOVE_UP,223988.0
2,Reggaetón Lento (Bailemos),3,2017-01-01,CNCO,https://open.spotify.com/track/3AEZUABDXNtecAO...,Argentina,top200,MOVE_DOWN,210943.0
3,Safari,4,2017-01-01,"J Balvin, Pharrell Williams, BIA, Sky",https://open.spotify.com/track/6rQSrBHf7HlZjtc...,Argentina,top200,SAME_POSITION,173865.0
4,Shaky Shaky,5,2017-01-01,Daddy Yankee,https://open.spotify.com/track/58IL315gMSTD37D...,Argentina,top200,MOVE_UP,153956.0


In [ ]:
# Create a separate column for Year
df_charts['Year']=df_charts['date'].str.slice(stop=4)

# Cast 'Year' as Int
df_charts['Year'] = df_charts['Year'].astype(int)

# Show datatypes 
df_charts.dtypes

#remove ID, trend and streams columns
df_charts.drop(['url', 'trend', 'streams'], axis=1, inplace=True)

In [ ]:
# Number of record per year
df_charts.groupby(by='Year').count()



,title,rank,date,artist,region,chart
Year,,,,,,
2017,4401145,4401145,4401145,4401145,4401145,4401145
2018,4863016,4863016,4863016,4863016,4863016,4863016
2019,5412658,5412669,5412669,5412669,5412669,5412669
2020,5793148,5793148,5793148,5793130,5793148,5793148
2021,5703536,5703536,5703536,5703536,5703536,5703536


In [ ]:
# Remore records with year 2017 and 2018
df_charts=df_charts[df_charts['Year'] > 2018]

In [ ]:
df_charts.groupby(by='Year').count()

,title,rank,date,artist,region,chart
Year,,,,,,
2019,5412658,5412669,5412669,5412669,5412669,5412669
2020,5793148,5793148,5793148,5793130,5793148,5793148
2021,5703536,5703536,5703536,5703536,5703536,5703536
